In [1]:
import sys
# sys.path.append("/home/riccardo/mmdetection3d")
# sys.path.append("/root/workspace/mmdeploy")
from mmdet3d.apis import LidarDet3DInferencer
from mmdeploy.apis.inference import inference_model
from mmdeploy.apis.inference import get_model
import os
import torch
import os
import numpy as np
import time
from mmdet3d.datasets.transforms.loading import (LoadAnnotations3D,
                                                LoadPointsFromFile)
from mmdet3d.models.data_preprocessors.data_preprocessor import \
    Det3DDataPreprocessor
import onnxruntime as ort
from mmdet3d.datasets.transforms.formating import Pack3DDetInputs

In [3]:
# initialize inferencer 
# execute >> pip install -v -e . in mmdetection3d folder 
# CHANGE THE center_mode in loca_visualizer accordingly (KITTI -> center_mode = 'lidar_bottom', CUSTOM -> center_mode = whatever)

# inferencer = LidarDet3DInferencer('pointpillars_hv_secfpn_8xb6-160e_kitti-3d-car')
# inferencer = LidarDet3DInferencer('pointpillars_donaset_container-car')
# inferencer = LidarDet3DInferencer('pointpillars_kittilidar-car') 
# inferencer = LidarDet3DInferencer("pointpillars_hv_secfpn_8xb6-160e_lidar_vegas_iris-3d-car",scope="mmdet3d")
# inferencer = LidarDet3DInferencer("pointpillars_lidar_vegas_iris-car_360",scope="mmdet3d")
# inferencer = LidarDet3DInferencer("pointpillars_hydris_car",scope="mmdet3d")
# inferencer = LidarDet3DInferencer("pointpillars_hydris_car",scope="mmdet3d")
inferencer = LidarDet3DInferencer("pointpillars_hydra_custom_3000",scope="mmdet3d")


Loads checkpoint by local backend from path: /home/riccardo/LidarObjDetection2/mmdetection3d/work_dirs/pointpillars_hydra_custom/iter_3000.pth
07/08 16:59:51 - mmengine - WARNING - Failed to search registry with scope "mmdet3d" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet3d" is a correct scope, or whether the registry is initialized.
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/usr/local/lib/python3.10/dist-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


# Prediction Tests

In [5]:
dataset_dir = "/mnt/ssd/Datasets/HYDRA/2023-01-07_14-58/TrainingDataset_normalized"

### Create Validation

In [7]:
import os

def copy_bin_files(source_txt, bin_folder_path,destination_folder):


  # Check if source text file exists
  if not os.path.exists(source_txt):
    print(f"Error: Source text file '{source_txt}' does not exist.")
    return

  # Check if destination folder exists, create it if not
  if not os.path.exists(destination_folder):
    try:
      os.makedirs(destination_folder)
      print(f"Destination folder '{destination_folder}' created.")
    except OSError as e:
      print(f"Error creating destination folder: {e}")
      return

  # Open the source text file
  try:
    with open(source_txt, 'r') as file:
      lines = file.readlines()
  except OSError as e:
    print(f"Error opening source text file: {e}")
    return

  # Process each line in the text file
  for line in lines:
    # Remove leading/trailing whitespace and newline character
    filename = line.strip() +".bin"

    # Check if the filename ends with .bin
    if not filename.endswith('.bin'):
      print(f"Warning: Skipping non-binary file '{filename}'.")
      continue

    # Construct source and destination paths
    source_path = os.path.join(bin_folder_path, filename)
    dest_path = os.path.join(destination_folder, filename)

    # Check if source file exists
    if not os.path.exists(source_path):
      print(f"Warning: Source file '{source_path}' does not exist. Skipping...")
      continue

    # Copy the file
    try:
      with open(source_path, 'rb') as source, open(dest_path, 'wb') as destination:
        for chunk in iter(lambda: source.read(1024), b''):
          destination.write(chunk)
      print(f"Copied '{filename}' to '{destination_folder}'.")
    except OSError as e:
      print(f"Error copying file '{filename}': {e}")

# Example usage
source_txt_path = dataset_dir +"/ImageSets/val.txt"  # Replace with your actual path
destination_folder_path = dataset_dir +"/validation"  # Replace with your actual path
bin_folder_path = dataset_dir +"/training/velodyne"
copy_bin_files(source_txt_path, bin_folder_path, destination_folder_path)


Destination folder '/mnt/ssd/Datasets/HYDRA/2023-01-07_14-58/TrainingDataset_normalized/validation' created.
Copied '000428.bin' to '/mnt/ssd/Datasets/HYDRA/2023-01-07_14-58/TrainingDataset_normalized/validation'.
Copied '000083.bin' to '/mnt/ssd/Datasets/HYDRA/2023-01-07_14-58/TrainingDataset_normalized/validation'.
Copied '000363.bin' to '/mnt/ssd/Datasets/HYDRA/2023-01-07_14-58/TrainingDataset_normalized/validation'.
Copied '000111.bin' to '/mnt/ssd/Datasets/HYDRA/2023-01-07_14-58/TrainingDataset_normalized/validation'.
Copied '000443.bin' to '/mnt/ssd/Datasets/HYDRA/2023-01-07_14-58/TrainingDataset_normalized/validation'.
Copied '000437.bin' to '/mnt/ssd/Datasets/HYDRA/2023-01-07_14-58/TrainingDataset_normalized/validation'.
Copied '000330.bin' to '/mnt/ssd/Datasets/HYDRA/2023-01-07_14-58/TrainingDataset_normalized/validation'.
Copied '000439.bin' to '/mnt/ssd/Datasets/HYDRA/2023-01-07_14-58/TrainingDataset_normalized/validation'.
Copied '000421.bin' to '/mnt/ssd/Datasets/HYDRA/202

### Inference

In [ ]:
inferencer = LidarDet3DInferencer("pointpillars_hydra_custom_3000",scope="mmdet3d")
pcl = os.path.join(dataset_dir,"validation")
for file in os.listdir(pcl):
    print(file)
    inputs = dict(points=os.path.join(pcl,file))
    inf_res = inferencer(inputs, show=False,pred_score_thr=0.2, batch_size=1)
    break
inf_res

In [6]:
# inference
# pcl = '../data/Dataset_Lidar_20240111_vegas_ontrack_01/validation'
pcl = os.path.join(dataset_dir,"validation")
# pcl = os.path.join(dataset_dir,"training","velodyne")
# pcl = '../data/Hydra/training/velodyne/000020.bin'
# pcl = '../data/DonaSet/training/velodyne/'
# # pcl = './data/falcon/falcon1.bin'
# inputs = dict(points=pcl)
# inf_res = inferencer(inputs, show=True,pred_score_thr=0.3, batch_size=1)
# inf_res

for file in os.listdir(pcl):
    print(file)
    inputs = dict(points=os.path.join(pcl,file))
    inf_res = inferencer(inputs, show=True,pred_score_thr=0.2, batch_size=1)

000413.bin


/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
000228.bin
[Open3D WARNING] [ViewControl] ConvertFromPinholeCameraParameters() failed because window height and width do not match.
000241.bin
[Open3D WARNING] [ViewControl] ConvertFromPinholeCameraParameters() failed because window height and width do not match.
000163.bin
[Open3D WARNING] [ViewControl] ConvertFromPinholeCameraParameters() failed because window height and width do not match.
000040.bin
[Open3D WARNING] [ViewControl] ConvertFromPinholeCameraParameters() failed because window height and width do not match.
000335.bin
[Open3D WARNING] [ViewControl] ConvertFromPinholeCameraParameters() failed because window height and width do not match.
000247.bin
[Open3D WARNING] [ViewControl] ConvertFromPinholeCameraParameters() failed because window height and width do not match.
[Open3D WARNING] GLFW Error: The GLFW library is not initialized

SystemExit: 0

/home/riccardo/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import time
start = time.time()
inferencer(inputs, show=False,pred_score_thr=0.3,batch_size=1)
end = time.time()
print("Time Taken {} ms".format((end-start)*1000))

In [ ]:
from mmdet3d.datasets.transforms.loading import (LoadAnnotations3D,
                                                 LoadPointsFromFile)
from mmdet3d.models.data_preprocessors.data_preprocessor import \
    Det3DDataPreprocessor
from mmdet3d.datasets.transforms.formating import Pack3DDetInputs
import os 

def prepare_input(inputs,batch_size=1):
    model_inputs = {}
    model_inputs["inputs"] = {}
    model_inputs["inputs"]["points"] = []
    max = batch_size

    files = []
    if not os.path.isdir(inputs["points"]):
        files.append(inputs["points"])
    else:
        files = [inputs["points"] + s for s in os.listdir(inputs["points"])]
    for i,sample_path in enumerate(files):
        inp = {}
        inp["lidar_points"] = {}
        inp["lidar_points"]["lidar_path"] = sample_path


        loader = LoadPointsFromFile(coord_type='LIDAR',load_dim=4,use_dim=4)
        packer = Pack3DDetInputs(keys=['points'])

        voxel_size = [0.16, 0.16, 4]
        preprocessor = Det3DDataPreprocessor(
            voxel=True,
            voxel_layer=dict(
                max_num_points=32,  # max_points_per_voxel
                point_cloud_range=[0, -39.68, -3, 69.12, 39.68, 1],
                voxel_size=voxel_size,
                max_voxels=(16000, 40000)))

        input = loader(inp)
        input = packer(input)
        # input["inputs"]["points"] = input["inputs"]["points"].cuda()
        model_inputs["inputs"]["points"].append(input["inputs"]["points"])
        if i == max-1:
            print("Batch Size!")
            break
    prep_input = preprocessor(model_inputs)
    prep_input["inputs"]["voxels"]["voxels"] = prep_input["inputs"]["voxels"]["voxels"].cuda()
    return prep_input

In [ ]:
prep_input = prepare_input({"points": pcl}, batch_size=1)

In [ ]:
model = inferencer.model


In [ ]:
import time 
import torch

torch.manual_seed(42)
s = time.time()

prep_input = prepare_input({"points": pcl}, batch_size=1) # Preprocess
res = model(prep_input["inputs"],prep_input["data_samples"], mode="tensor") # Forward
preds  = model.bbox_head.predict_by_feat(*res) # Postprocess

print("Time: {} ms".format((time.time()-s)*1000))
print(preds)

In [ ]:
inf_res = inferencer(inputs, show=False,pred_score_thr=0.3, batch_size=4)
inf_res

Convert Manually the tensor to predictions

In [ ]:
from mmdet3d.models.dense_heads.anchor3d_head import Anchor3DHead

# bbox_head=dict(
#     type='Anchor3DHead',
#     num_classes=3,
#     in_channels=384,
#     feat_channels=384,
#     use_direction_classifier=True,
#     assign_per_class=True,
#     anchor_generator=dict(
#         type='AlignedAnchor3DRangeGenerator',
#         ranges=[
#             [0, -39.68, -0.6, 69.12, 39.68, -0.6],
#             [0, -39.68, -0.6, 69.12, 39.68, -0.6],
#             [0, -39.68, -1.78, 69.12, 39.68, -1.78],
#         ],
#         sizes=[[0.8, 0.6, 1.73], [1.76, 0.6, 1.73], [3.9, 1.6, 1.56]],
#         rotations=[0, 1.57],
#         reshape_out=False),
#     diff_rad_by_sin=True,
#     bbox_coder=dict(type='DeltaXYZWLHRBBoxCoder')
# )
# model.bbox_head
head = Anchor3DHead(
    num_classes=1,
    in_channels=384,
    feat_channels=384,
    use_direction_classifier=True,
    assign_per_class=True,
    anchor_generator=dict(
                type='AlignedAnchor3DRangeGenerator',
                ranges=[[0, -39.68, -1.78, 69.12, 39.68, -1.78]],
                sizes=[[3.9, 1.6, 1.56]],
                rotations=[0, 1.57],
                reshape_out=True),
    diff_rad_by_sin=True,
    bbox_coder=dict(type='DeltaXYZWLHRBBoxCoder'),
    test_cfg=model.test_cfg
    )
# preds  = head.predict_by_feat(*res)
preds  = model.bbox_head.predict_by_feat(*res)
preds

In [ ]:
preds[0]["bboxes_3d"],pred_res[0]["bboxes_3d"],inf_res["predictions"][0]["bboxes_3d"]

# Model Analysis

In [ ]:
model = inferencer.model
pcl_path = os.path.join(dataset_dir, "training/velodyne/000020.bin")
model_path = "deployed_models/hydris/end2end.onnx"
model_cfg = "../configs/pointpillars/pointpillars_hydris_car.py"


In [ ]:
from mmengine.config import Config

def prepare_input(inputs,batch_size=1, model_cfg = None, device="cuda",data_samples=False, deterministic=True):
    model_inputs = {}
    model_inputs["inputs"] = {}
    model_inputs["inputs"]["points"] = []
    if data_samples:
        model_inputs["data_samples"] = []
    max = batch_size

    files = []
    if not os.path.isdir(inputs["points"]):
        files.append(inputs["points"])
    else:
        files = [inputs["points"] + s for s in os.listdir(inputs["points"])]
    for i,sample_path in enumerate(files):
        inp = {}
        inp["lidar_points"] = {}
        inp["lidar_points"]["lidar_path"] = sample_path


        loader = LoadPointsFromFile(coord_type='LIDAR',load_dim=4,use_dim=4)
        packer = Pack3DDetInputs(keys=['points'])

        cfg = Config.fromfile(model_cfg)
        cfg["model"]["data_preprocessor"].pop("type")
        preprocessor = Det3DDataPreprocessor(**cfg["model"]["data_preprocessor"])
        if not deterministic:
            preprocessor.cuda()
        input = loader(inp)
        input = packer(input)
        # input["inputs"]["points"] = input["inputs"]["points"].cuda()
        model_inputs["inputs"]["points"].append(input["inputs"]["points"])
        if data_samples:
            model_inputs["data_samples"].append(input["data_samples"])
        if i == max-1:
            # print("Batch Size!")
            break
    prep_input = preprocessor(model_inputs)
    if device == "cuda":
        prep_input["inputs"]["voxels"]["voxels"] = prep_input["inputs"]["voxels"]["voxels"].cuda()
    return prep_input

## Inference time comparison

### Pytorch CPU

In [ ]:
s0 = time.time()
model.cpu()
inputs = prepare_input({"points": pcl_path}, batch_size=1, model_cfg=model_cfg,device = "cpu")["inputs"]

s1 = time.time()
val_output = model(inputs,mode="tensor")
s2 = time.time()
preds  = model.bbox_head.predict_by_feat(*val_output)

print("Pre: {} ms".format((s1-s0)*1000))
print("Inf: {} ms".format((s2-s1)*1000))
print("Post: {} ms".format((time.time()-s2)*1000))
print("Total: {} ms".format((time.time()-s1)*1000))

In [ ]:
torch.tensor(inf_res["predictions"][0]["bboxes_3d"][0]) -  list(preds[0].bboxes_3d.cpu())[0]

### Pytorch GPU

In [ ]:
s0 = time.time()
model.cuda()
inputs = prepare_input({"points": pcl_path}, batch_size=1, model_cfg =  model_cfg, deterministic=True)["inputs"]

s1 = time.time()
val_output = model(inputs,mode="tensor")
s2 = time.time()
preds  = model.bbox_head.predict_by_feat(*val_output)

print("Pre: {} ms".format((s1-s0)*1000))
print("Inf: {} ms".format((s2-s1)*1000))
print("Post: {} ms".format((time.time()-s2)*1000))
print("Total: {} ms".format((time.time()-s1)*1000))

In [ ]:
torch.tensor(inf_res["predictions"][0]["bboxes_3d"][0]) -  list(preds[0].bboxes_3d.cpu())[0]

### TensorRT Provider (ONNX Runtime)

In [ ]:
ort_sess = ort.InferenceSession(model_path,providers=['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider'])

In [ ]:
s0= time.time()
inputs = prepare_input({"points": pcl_path}, batch_size=1, model_cfg = model_cfg, device="cuda")["inputs"]

onnx_inputs = {'voxels': inputs["voxels"]["voxels"].cpu().numpy().astype(np.float32), 'coors':inputs["voxels"]["coors"].cpu().numpy(),'num_points':inputs["voxels"]["num_points"].cpu().numpy()}

s1 = time.time()
outputs = ort_sess.run(None, onnx_inputs)
s2 = time.time()
outputs = [arr.astype(np.float32) for arr in outputs]
preds  = model.bbox_head.predict_by_feat(*outputs)
print("Pre: {} ms".format((s1-s0)*1000))
print("Inf: {} ms".format((s2-s1)*1000))
print("Post: {} ms".format((time.time()-s2)*1000))
print("Total: {} ms".format((time.time()-s0)*1000))

In [ ]:
torch.tensor(inf_res["predictions"][0]["bboxes_3d"][0]) -  list(preds[0].bboxes_3d.cpu())[0]

### TensorRT Provider (ONNX Runtime) - FP16

In [ ]:
!pip install onnxconverter_common

In [ ]:
!pip install onnxmltools

In [ ]:
from onnx import load
from onnxmltools.utils.float16_converter import convert_float_to_float16
from onnx import save
onnx_model = load(model_path)
new_onnx_model = convert_float_to_float16(onnx_model)
save(new_onnx_model,"deployed_models/hydra/end2end_fp16.onnx")

In [ ]:
ort_sess = ort.InferenceSession("deployed_models/hydra/end2end_fp16.onnx",providers=['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider'])

In [ ]:
s0= time.time()
inputs = prepare_input({"points": pcl_path}, batch_size=1, model_cfg = model_cfg,device="cuda")["inputs"]

onnx_inputs = {'voxels': inputs["voxels"]["voxels"].cpu().numpy().astype(np.float16), 'coors':inputs["voxels"]["coors"].cpu().numpy(),'num_points':inputs["voxels"]["num_points"].cpu().numpy()}

s1 = time.time()
outputs = ort_sess.run(None, onnx_inputs)
s2 = time.time()
outputs = [arr.astype(np.float32) for arr in outputs]
preds  = model.bbox_head.predict_by_feat(*outputs)
print("Pre: {} ms".format((s1-s0)*1000))
print("Inf: {} ms".format((s2-s1)*1000))
print("Post: {} ms".format((time.time()-s2)*1000))
print("Total: {} ms".format((time.time()-s0)*1000))

In [ ]:
torch.tensor(inf_res["predictions"][0]["bboxes_3d"][0]) -  list(preds[0].bboxes_3d.cpu())[0]

### CUDA Provider  (ONNX Runtime)

In [ ]:
ort_sess = ort.InferenceSession(model_path,providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])

In [ ]:
inputs = prepare_input({"points": pcl_path}, batch_size=1)["inputs"]
onnx_inputs = {'voxels': inputs["voxels"]["voxels"].cpu().numpy(), 'coors':inputs["voxels"]["coors"].cpu().numpy(),'num_points':inputs["voxels"]["num_points"].cpu().numpy()}

s1 = time.time()
outputs = ort_sess.run(None, onnx_inputs)
s2 = time.time()
preds  = model.bbox_head.predict_by_feat(*outputs)
print("Inf: {} ms".format((s2-s1)*1000))
print("Post: {} ms".format((time.time()-s2)*1000))
print("Total: {} ms".format((time.time()-s1)*1000))

### CPU Provider  (ONNX Runtime)

In [ ]:
ort_sess = ort.InferenceSession(model_path,providers=[ 'CPUExecutionProvider'])

In [ ]:
inputs = prepare_input({"points": pcl_path}, batch_size=1)["inputs"]
onnx_inputs = {'voxels': inputs["voxels"]["voxels"].cpu().numpy(), 'coors':inputs["voxels"]["coors"].cpu().numpy(),'num_points':inputs["voxels"]["num_points"].cpu().numpy()}

s1 = time.time()
outputs = ort_sess.run(None, onnx_inputs)
s2 = time.time()
preds  = model.bbox_head.predict_by_feat(*outputs)
print("Inf: {} ms".format((s2-s1)*1000))
print("Post: {} ms".format((time.time()-s2)*1000))
print("Total: {} ms".format((time.time()-s1)*1000))

###  TensorRT

In [4]:
from mmengine.config import Config

def prepare_input(inputs,batch_size=1, model_cfg = None, device="cuda",data_samples=False, preprocessor=None,deterministic=True):
    model_inputs = {}
    model_inputs["inputs"] = {}
    model_inputs["inputs"]["points"] = []
    if data_samples:
        model_inputs["data_samples"] = []
    max = batch_size

    files = []
    if not os.path.isdir(inputs["points"]):
        files.append(inputs["points"])
    else:
        files = [inputs["points"] + s for s in os.listdir(inputs["points"])]
    for i,sample_path in enumerate(files):
        inp = {}
        inp["lidar_points"] = {}
        inp["lidar_points"]["lidar_path"] = sample_path


        loader = LoadPointsFromFile(coord_type='LIDAR',load_dim=4,use_dim=4)
        packer = Pack3DDetInputs(keys=['points'])

        if not deterministic:
            preprocessor.cuda()
        input = loader(inp)
        input = packer(input)
        # input["inputs"]["points"] = input["inputs"]["points"].cuda()
        model_inputs["inputs"]["points"].append(input["inputs"]["points"])
        if data_samples:
            model_inputs["data_samples"].append(input["data_samples"])
        if i == max-1:
            # print("Batch Size!")
            break
    # print(preprocessor.device)
    s0 = time.time()
    prep_input = preprocessor(model_inputs)
    # print((time.time()-s0)*1000)
    if device == "cuda":
        prep_input["inputs"]["voxels"]["voxels"] = prep_input["inputs"]["voxels"]["voxels"].cuda()
    return prep_input

In [5]:

model_cfg = "../configs/pointpillars/pointpillars_hydris_car.py"
deploy_cfg = "../../mmdeploy/configs/mmdet3d/voxel-detection/voxel-detection_tensorrt_dynamic-kitti-32x4_fp16.py"
backend_files = "deployed_models/hydris/end2end_fp16.engine"
img = os.path.join(dataset_dir,"training/velodyne/000000.bin")
device = "cuda"

tensorrt_model = get_model(model_cfg,deploy_cfg,[backend_files],img,device)
preprocessor = tensorrt_model.data_preprocessor.cpu()


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
07/03 23:31:44 - mmengine - WARNING - Failed to search registry with scope "mmdet3d" in the "Codebases" registry tree. As a workaround, the current "Codebases" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet3d" is a correct scope, or whether the registry is initialized.
07/03 23:31:44 - mmengine - WARNING - Failed to search registry with scope "mmdet3d" in the "mmdet3d_tasks" registry tree. As a workaround, the current "mmdet3d_tasks" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet3d" is a correct scope, or whether the registry is initialized.
07/03 23:31:44 - mmengine - WARNING - Failed to search registry with scope "mmdet3d" in t

In [6]:
pcl_path =  os.path.join(dataset_dir,"training/velodyne/000020.bin")

s0 = time.time()
inputs = prepare_input({"points": pcl_path}, batch_size=1, preprocessor = preprocessor, data_samples=False)["inputs"]
# inputs = prepare_input_fast({"points": pcl_path}, batch_size=1, data_samples=False)["inputs"]

s1=time.time()
res = tensorrt_model(inputs, mode="tensor")
res["bbox_pred"][0] = res["bbox_pred"][0].to("cpu",non_blocking=True)
res["cls_score"][0] = res["cls_score"][0].to("cpu",non_blocking=True)
res["dir_cls_pred"][0] = res["dir_cls_pred"][0].to("cpu",non_blocking=True)
# combined_tensor = torch.cat([res["bbox_pred"][0], res["cls_score"][0], res["dir_cls_pred"][0]], dim=0)
# combined_cpu = combined_tensor.cpu()
s2 =time.time()
preds  = model.bbox_head.predict_by_feat(res["cls_score"],res["bbox_pred"],res["dir_cls_pred"])
print("Pre: {} ms".format((s1-s0)*1000))
print("Inf: {} ms".format((s2-s1)*1000))
print("Post: {} ms".format((time.time()-s2)*1000))
print("Total: {} ms".format((time.time()-s0)*1000))

NameError: name 'model' is not defined

### Without Inferencer

In [7]:
from mmengine.config import Config
from mmdet3d.models.dense_heads import Anchor3DHead

cfg = Config.fromfile(model_cfg)

cfg["model"]["bbox_head"].pop("type")
bbox_head = Anchor3DHead(**cfg["model"]["bbox_head"])
bbox_head.test_cfg = cfg["model"]["test_cfg"]

/root/workspace/mmdetection3d/mmdet3d/models/dense_heads/anchor3d_head.py:94: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  warnings.warn(


In [18]:
s0 = time.time()
inputs = prepare_input({"points": pcl_path}, batch_size=1, preprocessor=preprocessor,model_cfg=model_cfg,data_samples=False)["inputs"]

s1=time.time()
res = tensorrt_model(inputs, mode="tensor")
torch.cuda.synchronize()
s2 =time.time()
preds  = bbox_head.predict_by_feat(res["cls_score"],res["bbox_pred"],res["dir_cls_pred"])
print("Pre: {} ms".format((s1-s0)*1000))
print("Inf: {} ms".format((s2-s1)*1000))
print("Post: {} ms".format((time.time()-s2)*1000))
print("Total: {} ms".format((time.time()-s0)*1000))

Pre: 5.93256950378418 ms
Inf: 15.146017074584961 ms
Post: 3.1936168670654297 ms
Total: 24.293184280395508 ms


In [ ]:
torch.tensor(inf_res["predictions"][0]["bboxes_3d"][0]) -  list(preds[0].bboxes_3d.cpu())[0]

### TensorRT manual Inference

In [19]:
import tensorrt as trt
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit

def load_engine(engine_file_path):
    TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
    with open(engine_file_path, 'rb') as f:
        runtime = trt.Runtime(TRT_LOGGER)
        engine = runtime.deserialize_cuda_engine(f.read())
    return engine
def torch_dtype_from_trt(dtype: trt.DataType) -> torch.dtype:
    """Convert pytorch dtype to TensorRT dtype.

    Args:
        dtype (str.DataType): The data type in tensorrt.

    Returns:
        torch.dtype: The corresponding data type in torch.
    """

    if dtype == trt.bool:
        return torch.bool
    elif dtype == trt.int8:
        return torch.int8
    elif dtype == trt.int32:
        return torch.int32
    elif dtype == trt.float16:
        return torch.float16
    elif dtype == trt.float32:
        return torch.float32
    else:
        raise TypeError(f'{dtype} is not supported by torch')


def torch_device_from_trt(device: trt.TensorLocation):
    """Convert pytorch device to TensorRT device.

    Args:
        device (trt.TensorLocation): The device in tensorrt.
    Returns:
        torch.device: The corresponding device in torch.
    """
    if device == trt.TensorLocation.DEVICE:
        return torch.device('cuda')
    elif device == trt.TensorLocation.HOST:
        return torch.device('cpu')
    else:
        return TypeError(f'{device} is not supported by torch')



In [20]:
pcl_path =  os.path.join(dataset_dir,"training/velodyne/000020.bin")
backend_files = "deployed_models/hydris/end2end_fp16.engine"

In [21]:
n_inputs  = 3
n_outputs = 3
engine = load_engine(backend_files)
input_names = [engine.get_binding_name(i) for i in range(n_inputs)]
output_names = [engine.get_binding_name(i) for i in range(n_inputs,n_inputs+n_outputs)]

# Create context
context = engine.create_execution_context()

/tmp/ipykernel_322058/2476830997.py:4: DeprecationWarning: Use get_tensor_name instead.
  input_names = [engine.get_binding_name(i) for i in range(n_inputs)]
/tmp/ipykernel_322058/2476830997.py:5: DeprecationWarning: Use get_tensor_name instead.
  output_names = [engine.get_binding_name(i) for i in range(n_inputs,n_inputs+n_outputs)]


In [22]:
def do_inference(pcl_path):
    bindings = [None] * (n_inputs + n_outputs)

    # Transform Input
    inputs = prepare_input({"points": pcl_path}, batch_size=1, preprocessor=preprocessor,model_cfg=model_cfg,data_samples=False)["inputs"]
    preprocessed = inputs['voxels']
    input_dict = {
        'voxels': preprocessed['voxels'].to("cuda"),
        'num_points': preprocessed['num_points'].to("cuda"),
        'coors': preprocessed['coors'].to("cuda")
    }
    inputs = dict((name, data.contiguous().int() if data.dtype ==
                        torch.long else data.contiguous())
                        for name, data in input_dict.items())

    for input_name in input_names:
        idx = engine.get_binding_index(input_name)
        profile = engine.get_profile_shape(0, input_name)
        input_tensor = inputs[input_name]
        context.set_binding_shape(idx, tuple(input_tensor.shape))
        bindings[idx] = input_tensor.contiguous().data_ptr()

    # Perform inference
    outputs = {}
    for output_name in output_names:
        idx = engine.get_binding_index(output_name)

        dtype = torch_dtype_from_trt(engine.get_binding_dtype(idx))
        shape = tuple(context.get_binding_shape(idx))

        device = torch_device_from_trt(engine.get_location(idx))
        output = torch.empty(size=shape, dtype=dtype, device=device)
        outputs[output_name] = output
        bindings[idx] = output.contiguous().data_ptr()

    context.execute_async_v2(bindings,
                            torch.cuda.current_stream().cuda_stream)
    return outputs


In [34]:
s0=time.time()
do_inference(pcl_path)
torch.cuda.synchronize()
print("Took "+str((time.time()-s0)*1e3)+" ms")


Took 20.28489112854004 ms


/tmp/ipykernel_322058/1935469659.py:17: DeprecationWarning: Use get_tensor_name instead.
  idx = engine.get_binding_index(input_name)
/tmp/ipykernel_322058/1935469659.py:18: DeprecationWarning: Use get_tensor_profile_shape instead.
  profile = engine.get_profile_shape(0, input_name)
/tmp/ipykernel_322058/1935469659.py:20: DeprecationWarning: Use set_input_shape instead.
  context.set_binding_shape(idx, tuple(input_tensor.shape))
/tmp/ipykernel_322058/1935469659.py:26: DeprecationWarning: Use get_tensor_name instead.
  idx = engine.get_binding_index(output_name)
/tmp/ipykernel_322058/1935469659.py:28: DeprecationWarning: Use get_tensor_dtype instead.
  dtype = torch_dtype_from_trt(engine.get_binding_dtype(idx))
/tmp/ipykernel_322058/1935469659.py:29: DeprecationWarning: Use get_tensor_shape instead.
  shape = tuple(context.get_binding_shape(idx))
/tmp/ipykernel_322058/1935469659.py:31: DeprecationWarning: Use get_tensor_location instead.
  device = torch_device_from_trt(engine.get_locat

In [26]:
context.execute_async_v2(bindings,
                        torch.cuda.current_stream().cuda_stream)

: 

In [22]:
outputs["cls_score0"].shape

torch.Size([1, 2, 124, 436])

### TensorRT Fp16

In [ ]:


# model_cfg = "../configs/pointpillars/pointpillars_hydris_car.py"
deploy_cfg = "../../mmdeploy/configs/mmdet3d/voxel-detection/voxel-detection_tensorrt_dynamic-kitti-32x4_fp16.py"
backend_files = "deployed_models/hydra/end2end_fp16.engine"
img =  os.path.join(dataset_dir,"velodyne/000000.bin")
device = "cuda"

tensorrt_model = get_model(model_cfg,deploy_cfg,[backend_files],img,device)

In [ ]:
s0 = time.time()
inputs = prepare_input({"points": pcl_path}, batch_size=1,model_cfg=model_cfg,data_samples=False)["inputs"]

s1=time.time()
res = tensorrt_model(inputs, mode="tensor")
s2 =time.time()
preds  = model.bbox_head.predict_by_feat(res["cls_score"],res["bbox_pred"],res["dir_cls_pred"])
print("Pre: {} ms".format((s1-s0)*1000))
print("Inf: {} ms".format((s2-s1)*1000))
print("Post: {} ms".format((time.time()-s2)*1000))
print("Total: {} ms".format((time.time()-s0)*1000))

In [ ]:
torch.tensor(inf_res["predictions"][0]["bboxes_3d"][0]) -  list(preds[0].bboxes_3d.cpu())[0]

### Visualize

In [ ]:
inputs = dict(points=pcl)
inf_res = inferencer(inputs, show=True, pred_score_thr=0.3, batch_size=1)

In [ ]:
from mmdet3d.structures import Det3DDataSample

inputs = prepare_input({"points": pcl}, batch_size=1,model_cfg=model_cfg,data_samples=False)["inputs"]
preds = tensorrt_model(inputs, mode="predict")
ds = Det3DDataSample()
ds.pred_instances_3d = preds[0]

inferencer.visualize([{"points": pcl}], [ds],show=True,pred_score_thr=0.3)

In [ ]:
torch.tensor(inf_res["predictions"][0]["bboxes_3d"][0]) -  list(ds.pred_instances_3d.bboxes_3d.cpu())[0]

## Voxelization algorithm comparison

In [ ]:
from mmdet3d.models.data_preprocessors import Det3DDataPreprocessor


def prepare_input(inputs,batch_size=1, device="cuda",data_samples=False, deterministic=True):
    model_inputs = {}
    model_inputs["inputs"] = {}
    model_inputs["inputs"]["points"] = []
    if data_samples:
        model_inputs["data_samples"] = []
    max = batch_size

    files = []
    if not os.path.isdir(inputs["points"]):
        files.append(inputs["points"])
    else:
        files = [inputs["points"] + s for s in os.listdir(inputs["points"])]
    for i,sample_path in enumerate(files):
        inp = {}
        inp["lidar_points"] = {}
        inp["lidar_points"]["lidar_path"] = sample_path


        loader = LoadPointsFromFile(coord_type='LIDAR',load_dim=4,use_dim=4)
        packer = Pack3DDetInputs(keys=['points'])

        voxel_size = [0.16, 0.16, 4]
        preprocessor = Det3DDataPreprocessor(
            voxel=True,
            voxel_layer=dict(
                max_num_points=32,  # max_points_per_voxel
                point_cloud_range=[-60, -19.84, -3, 79.52, 19.84, 3],
                # point_cloud_range=[0, -39.68, -3, 69.12, 39.68, 3],
                voxel_size=voxel_size,
                max_voxels=(16000, 40000),
                deterministic=deterministic))
        if not deterministic:
            preprocessor.cuda()
        input = loader(inp)
        input = packer(input)
        # input["inputs"]["points"] = input["inputs"]["points"].cuda()
        model_inputs["inputs"]["points"].append(input["inputs"]["points"])
        if data_samples:
            model_inputs["data_samples"].append(input["data_samples"])
        if i == max-1:
            # print("Batch Size!")
            break
    prep_input = preprocessor(model_inputs)
    if device == "cuda":
        prep_input["inputs"]["voxels"]["voxels"] = prep_input["inputs"]["voxels"]["voxels"].cuda()
    return prep_input

In [ ]:
s0 = time.time()
inputs = prepare_input({"points": pcl_path}, batch_size=1,data_samples=False, deterministic=False, device="cuda")["inputs"]
print("Non deterministic voxel -> " +str((time.time()-s0)*1000))

s0 = time.time()
inputs = prepare_input({"points": pcl_path}, batch_size=1,data_samples=False, deterministic=True, device="cuda")["inputs"]
print("Deterministic voxel -> " +str((time.time()-s0)*1000))
